# scVI

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>).
- **Affiliation:** Helmholtz Munich, Institute of Computational Biology (ICB), Talavera-López Lab
- **Date of Creation:** 05.01.2023
- **Date of Last Modification:** 29.03.2023

- The scVI source code is available at https://github.com/scverse/scvi-tools.
- The corresponding publication is "Lopez, R., Regier, J., Cole, M. B., Jordan, M. I. & Yosef, N. Deep generative modeling for single-cell transcriptomics. Nat. Methods 15, 1053–1058 (2018)".
- The workflow of this notebook follows the tutorial from https://docs.scvi-tools.org/en/stable/tutorials/notebooks/harmonization.html.
- The authors use raw counts as input to scVI. Therefore, we also use raw counts (stored in adata.layers["counts"]).

## 1. Setup

### 1.1 Import Libraries

In [17]:
import os
import time
from datetime import datetime

import anndata as ad
import scvi
import scanpy as sc
import scipy.sparse as sp
import squidpy as sq
import matplotlib.pyplot as plt
import numpy as np

### 1.2 Define Parameters

In [23]:
model_name = "scvi"
latent_key = f"{model_name}_latent"
mapping_entity_key = "reference"
condition_key = "batch"
counts_key = "counts"
leiden_resolution = 0.5 # used for Leiden clustering of latent space
random_seed = 0 # used for Leiden clustering

### 1.3 Run Notebook Setup

In [11]:
sc.set_figure_params(figsize=(6, 6))

In [12]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [13]:
srt_data_gold_folder_path = "../../datasets/srt_data/gold/"
figure_folder_path = f"../../figures"

## 2. scVI Model

### 2.1 Define Training Function

In [29]:
def train_scvi_models(dataset,
                      reference_batches,
                      cell_type_key,
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=10,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20],
                      plot_latent_umaps=False):
    # Configure figure folder path
    dataset_figure_folder_path = f"{figure_folder_path}/{dataset}/sample_integration_method_benchmarking/" \
                                 f"{model_name}/{current_timestamp}"
    os.makedirs(dataset_figure_folder_path, exist_ok=True)
    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:  
        adata_batch_list = []
        if reference_batches is not None:
            for batch in reference_batches:
                adata_batch = ad.read_h5ad(
                    f"{srt_data_gold_folder_path}/{dataset}_{batch}.h5ad")
                adata_batch.obs[mapping_entity_key] = "reference"
                adata_batch_list.append(adata_batch)
            adata_original = ad.concat(adata_batch_list, join="inner")
        else:
            adata_original = ad.read_h5ad(f"{srt_data_gold_folder_path}/{dataset}.h5ad")

        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        adata_new.obs[condition_key] = adata_original.obs[condition_key]
        adata_new.obs[mapping_entity_key] = adata_original.obs[mapping_entity_key] 
        del(adata_original)
        
    model_seeds = list(range(10))
    for run_number, n_neighbors in zip(np.arange(n_start_run, n_end_run+1), n_neighbor_list):
        # n_neighbors is here only used for the latent neighbor graph construction used for
        # UMAP generation and clustering as scVI is not a spatial method
        
        # Load data
        adata_batch_list = []
        if reference_batches is not None:
            for batch in reference_batches:
                print(f"Processing batch {batch}...")
                print("Loading data...")
                adata_batch = ad.read_h5ad(
                    f"{srt_data_gold_folder_path}/{dataset}_{batch}.h5ad")
                adata_batch_list.append(adata_batch)
            adata = ad.concat(adata_batch_list, join="inner")
        else:
            adata = ad.read_h5ad(f"{srt_data_gold_folder_path}/{dataset}.h5ad")
        
        start_time = time.time()
        
        scvi.settings.seed = model_seeds[run_number-1]

        # Setup adata
        scvi.model.SCVI.setup_anndata(adata,
                                      layer=counts_key,
                                      batch_key=condition_key)

        # Initialize model
        # Use hyperparams that provenly work well on integration tasks
        model = scvi.model.SCVI(adata,
                                n_layers=2,
                                n_latent=30,
                                gene_likelihood="nb")

        # Train model
        model.train()

        # Store latent representation
        adata.obsm[latent_key] = model.get_latent_representation()
        
        # Measure time for model training
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Duration of model training in run {run_number}: "
              f"{int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds.")
        adata_new.uns[f"{model_name}_model_training_duration_run{run_number}"] = (
            elapsed_time)
        
        if plot_latent_umaps:
            # Use scVI latent space for UMAP generation
            sc.pp.neighbors(adata,
                            use_rep=latent_key,
                            n_neighbors=n_neighbors)
            sc.tl.umap(adata)
            fig = sc.pl.umap(adata,
                             color=[cell_type_key],
                             title="Latent Space with Cell Types: scVI",
                             return_fig=True)
            fig.savefig(f"{dataset_figure_folder_path}/latent_{model_name}"
                        f"_cell_types_run{run_number}.png",
                        bbox_inches="tight")

            # Compute latent Leiden clustering
            sc.tl.leiden(adata=adata,
                         resolution=leiden_resolution,
                         random_state=random_seed,
                         key_added=f"latent_{model_name}_leiden_{str(leiden_resolution)}")

            # Create subplot of latent Leiden cluster annotations in physical and latent space
            fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 12))
            title = fig.suptitle(t="Latent and Physical Space with Leiden Clusters: scVI")
            sc.pl.umap(adata=adata,
                       color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                       title=f"Latent Space with Leiden Clusters",
                       ax=axs[0],
                       show=False)
            sq.pl.spatial_scatter(adata=adata,
                                  color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                                  title=f"Physical Space with Leiden Clusters",
                                  shape=None,
                                  ax=axs[1])

            # Create and position shared legend
            handles, labels = axs[0].get_legend_handles_labels()
            lgd = fig.legend(handles, labels, bbox_to_anchor=(1.25, 0.9185))
            axs[0].get_legend().remove()
            axs[1].get_legend().remove()

            # Adjust, save and display plot
            plt.subplots_adjust(wspace=0, hspace=0.2)
            fig.savefig(f"{dataset_figure_folder_path}/latent_physical_comparison_"
                        f"{model_name}_run{run_number}.png",
                        bbox_extra_artists=(lgd, title),
                        bbox_inches="tight")
            plt.show()
        
        # Store latent representation
        adata_new.obsm[latent_key + f"_run{run_number}"] = adata.obsm[latent_key]

        # Store intermediate adata to disk
        adata_new.write(f"{srt_data_gold_folder_path}/{dataset}_{model_name}_oneshot_integrated.h5ad")  

    # Store final adata to disk
    adata_new.write(f"{srt_data_gold_folder_path}/{dataset}_{model_name}_oneshot_integrated.h5ad") 

### 2.2 Train Models on Benchmarking Datasets

In [30]:
train_scvi_models(dataset="starmap_plus_mouse_cns",
                  reference_batches=["batch1_sample", "batch2_sample", "batch3_sample"],
                  cell_type_key="Main_molecular_cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

Global seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Processing batch batch1_sample...
Loading data...
Processing batch batch2_sample...
Loading data...
Processing batch batch3_sample...
Loading data...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [03:05<00:00,  2.12it/s, loss=269, v_num=1]

`Trainer.fit` stopped: `max_epochs=400` reached.


...

/home/cartalop/miniconda3/envs/autotalker/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Duration of model training in run 10: 0 hours, 0 minutes and 0 seconds.


Exception ignored in: <function Profiler.__del__ at 0x7fcf4c91eef0>
Traceback (most recent call last):
  File "/home/cartalop/miniconda3/envs/autotalker/lib/python3.10/site-packages/pytorch_lightning/profilers/profiler.py", line 144, in __del__
    def __del__(self) -> None:
KeyboardInterrupt: 


Epoch 7/400:   2%|█▋                                                                                                                 | 6/400 [00:03<03:54,  1.68it/s, loss=272, v_num=1]



KeyboardInterrupt



In [ ]:
# ~20% sample of the original dataset
train_scvi_models(dataset="starmap_plus_mouse_cns_sample",
                  cell_type_key="Main_molecular_cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])